# Prediksi Hilal

In [ ]:
# _jengkolrebus
# Juni, 2020
# Curup, Bengkulu

Koordinat kota:
- Yogyakarta: 7.7956 S, 110.3695 E, <a href="https://www.google.com/search?q=koordinat+yogyakarta&rlz=1C1CHBF_enID904ID904&oq=koordinat+yogyakarta&aqs=chrome..69i57.5458j0j7&sourceid=chrome&ie=UTF-8">sumber</a>
- Sabang: 5.892386 N, 95.323706 E, sumber: google maps
- Merauke: 8.499324 S, 140.404902 E, sumber: google maps

In [1]:
from skyfield import almanac
from skyfield.api import load, Topos
from skyfield.units import Angle

from datetime import datetime, timedelta
from pytz import timezone

from ipywidgets import widgets, interactive
from IPython.display import display, HTML

import pandas as pd
import calendar

In [2]:
jkt = timezone('Asia/Jakarta')
ts = load.timescale(builtin=True)
e = load('de421.bsp')


In [3]:
class Var:
    SAB = ['5.892386 N', '95.323706 E']
    YOG = ['7.7956 S','110.3695 E']
    MER = ['8.499324 S', '140.404902 E']

    KOTA = (SAB, YOG, MER)

In [10]:
class Find():
    t_konjungsi = []
    t_sunset = []
    t_moonset = []
    moon_alt = []
    moon_az = []
    sun_alt = []
    sun_az = []
    moon_astrometric = []
    sun_astrometric = []
    elong = []
    usia_bulan = []
    ir = []
    
    def __init__(self, t0, t1, topo):
        self.t0 = t0
        self.t1 = t1
        self.topo = Topos(topo[0], topo[1])
        print(self.topo)
        self.loc = e['earth'] + self.topo
        self.earth = e['earth']
        self.moon = e['moon']
        self.sun = e['sun']
        
    # Metode untuk membandingkan dengan Imkan Rukyat
    def imkanRukyat(self):
        alt = self.moon_alt
        elong = self.elong
        usia_bulan = self.usia_bulan
        for i, j, k in zip(alt, elong, usia_bulan):
            if(i.degrees >= 2 and j.degrees >= 3 and (timedelta.total_seconds(k)/3600) > 8):
                self.ir.append(u'\u2714')
            else:
                self.ir.append(u'\u2718')
        
    def moonset(self, t): 
        sunset = t
        t = t.utc
        t0 = ts.utc(t[0], t[1], t[2], t[3], t[4], t[5])
        t1 = ts.utc(t[0], t[1], t[2]+1, t[3]+1, t[4], t[5])
        f = almanac.risings_and_settings(e, self.moon, self.topo)
        t, y = almanac.find_discrete(t0, t1, f)

        for ti, yi in zip(t, y):
            if(yi==False):
                self.t_moonset.append(ti.astimezone(jkt))
            else:
                pass
    
    def pos(self, t, obj):
        astrometric = self.loc.at(t).observe(e[obj])
        alt, az, d = astrometric.apparent().altaz()
        if(obj == 'moon'):
            self.moon_astrometric.append(astrometric)
            self.moon_alt.append(alt)
            self.moon_az.append(az)
        else:
            self.sun_astrometric.append(astrometric)
            self.sun_alt.append(alt)
            self.sun_az.append(az)
    
    def sunset(self, t):
        t = t.utc
        t0 = ts.utc(t[0], t[1], t[2], t[3], t[4], t[5])
        t1 = ts.utc(t[0], t[1], t[2]+1, t[3], t[4], t[5])
        f = almanac.sunrise_sunset(e, self.topo)
        t, y = almanac.find_discrete(t0, t1, f)
        for ti, yi in zip(t, y):
            if(yi == False):
                self.t_sunset.append(ti.astimezone(jkt))
                self.pos(ti, 'moon')
                self.pos(ti, 'sun')
    
    
    def konjungsi(self):
        t0 = ts.utc(self.t0)
        t1 = ts.utc(self.t1)
        f = almanac.oppositions_conjunctions(e, self.moon)
        t, y = almanac.find_discrete(t0, t1, f)
        for ti, yi in zip(t, y):
            if(yi == 1):
                self.t_konjungsi.append(ti.astimezone(jkt))
                self.sunset(ti)
                self.moonset(ti)
            else:
                pass
            
    def newMoon(self):
        t0 = ts.utc(self.t0)
        t1 = ts.utc(self.t1)
        f = almanac.moon_phases(e)
        t, y = almanac.find_discrete(t0, t1, f)
        for ti, yi in zip(t, y):
            if(yi == 1):
                self.t_konjungsi.append(ti.astimezone(jkt))
                self.sunset(ti)
                self.moonset(ti)
            else:
                pass
    

In [11]:
def result(t0, t1, i):
    print()
    Find.moon_astrometric.clear()
    Find.t_konjungsi.clear()
    Find.t_sunset.clear()
    Find.t_moonset.clear()
    Find.moon_alt.clear()
    Find.moon_az.clear()
    Find.moon_astrometric.clear()
    Find.sun_alt.clear()
    Find.sun_az.clear()
    Find.sun_astrometric.clear()
    Find.sun_alt.clear()
    Find.elong.clear()
    Find.usia_bulan.clear()
    Find.ir.clear()
    
    f = Find(t0, t1, i)
    
    # Waktu Konjungsi
    f.newMoon()
    
    # Elongasi
    for i, j in zip(f.moon_astrometric, f.sun_astrometric):
        elong = i.separation_from(j)
        f.elong.append(elong)
    # Usia Bulan
    for i, j in zip(f.t_konjungsi, f.t_moonset):
        dt = j - i
#         print(type(dt))
        f.usia_bulan.append(dt)
            
    # Imkan Rukyat
    f.imkanRukyat()
    
    table = list(zip(f.t_konjungsi, f.t_sunset, 
                     f.moon_alt, f.moon_az, 
                     f.sun_az, f.elong, 
                     f.usia_bulan, f.ir))
    
    df = pd.DataFrame(table, columns=['Waktu Konjungsi', 'Waktu Sunset', 
                                     'Alt Bulan', 'Az Bulan', 
                                     'Az Matahari', 'Elongasi', 
                                      'Usia Bulan', 'Imkan Rukyat'])
    df.index+=1
    display(df)
    print()
        

def repeat(t0, t1):
    for i in Var.KOTA:
        result(t0, t1, i)

In [12]:
hari_ini = datetime.now().date()
days_in_month = calendar.monthrange(hari_ini.year, hari_ini.month)[1]
bulan_depan = hari_ini + timedelta(days_in_month)

dari = widgets.DatePicker(value=hari_ini,
    description='Dari:',
    disabled=False
)
sampai = widgets.DatePicker(value=bulan_depan,
    description='Sampai:',
    disabled=False
)

interactive(repeat, t0=dari, t1=sampai)

interactive(children=(DatePicker(value=datetime.date(2021, 1, 5), description='Dari:'), DatePicker(value=datet…